# Scraping google images to create a multi-label dataset

The previous model that we trained with images collected from open-source datasets was limited by its size and its single label class. We'll run a scraping tool to download more images from Google Image for different classes of military vehicles. 

## Defining labels

It's hard to find a unified taxonomy for military vehicles. We'll try to define large class labels by using Wikipedia's [Military vehicles by type](https://en.wikipedia.org/wiki/Category:Military_vehicles_by_type) category. Model names can be found in this list of [modern armoured fighting vehicles](https://en.wikipedia.org/wiki/List_of_modern_armoured_fighting_vehicles)

- **Armoured fighting vehicle (AFV)** is an armed combat vehicle protected by armour, generally combining operational mobility with offensive and defensive capabilities. AFVs can be wheeled or tracked. Examples of AFVs are tanks, armoured cars, assault guns, self-propelled guns, infantry fighting vehicles (IFV), and armoured personnel carriers (APC).
- **Armoured personnel carrier (APC)** is a broad type of armoured military vehicle designed to transport personnel and equipment in combat zones.
- **Military engineering vehicle (MEV)** is a vehicle built for construction work or for the transportation of combat engineers on the battlefield.
- **Light armoured vehicle (LAV) (incl. Reconnaissance vehicle - RV)** is the lightest weight class military vehicle category. A Jeep-like four-wheel drive vehicle for military use, with light or no armour. **Reconnaissance vehicle (RV)** is a military vehicle used for forward reconnaissance. Both tracked and wheeled reconnaissance vehicles are in service.

Based on these categories, we can define some search terms.

In [ ]:
AFV = [
    "AFV Lynx",
    "Boxer AFV",
    "ZTZ-99",
    "ZTZ-96",
    "VT-4",
    "ZBD-04",
    "Leclerc tank",
    "AMX 10 RC",
    "Leopard tank",
    "T-90",
    "T-72",
    "challenger tank",
    "M1 abrams",
]
APC = [
    "AMX-10P",
    "VAB",
    "LAV III",
    "Berliet VXB",
    "Panhard VCR",
    "Didgori-3",
    "M113 APC",
    "AMPV",
    "VBTP-MR Guarani",
    "BTR-40",
    "BTR-60",
    "BTR-80",
    "TPZ Fuchs",
    "Bison APC",
    "ZBL-08",
    "fv103 spartan",
    "MRAP",
]
MEV = [
    "Engin blindé du génie",
    "ebg vulcain",
    "kodiak wisent armoured vehicle",
    "m728 cev",
    "terrier armoured vehicle",
    "imr-2 armoured vehicle",
]
LAV = [
    "LAV-25",
    "Iveco VM 90",
    "Panhard VBL",
    "Panhard AML",
    "Panhard ERC",
    "Humvee",
    "FV601 Saladin",
    "AMX-10 RC",
    "RG-32 Scout",
    "fv101 scorpion",
    "fv107 scimitar",
]

## Downloading images from google

Once we've defined our labels and search terms, we can download images from Google for each category. We'll create our dataset by downloading 50 images for each search term.

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
MAX_IMAGES_PER_TERM = 50

In [ ]:
from adomvi.scraper.google import GoogleImageScraper
from pathlib import Path

def worker_thread(klass, search_term):
    save_dir = Path(f"google/{klass}")
    scraper = GoogleImageScraper(
        save_dir,
        search_term,
        max_images=MAX_IMAGES_PER_TERM,
        min_resolution=(400, 300),
        max_resolution=(2048, 2048),
    )
    images = scraper.get_image_urls()
    scraper.save_images(images)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

with ThreadPoolExecutor(max_workers=2) as executor:
    executor.map(worker_thread, repeat("AFV"), AFV)

## Annotate the dataset

To annotate the dataset, use a tool like [CVAT](https://app.cvat.ai/).

## Load the dataset

We provide a sample annotated dataset with 4 classes (*AFV*, *APC*, *LAV* & *MEV*). You can download the dataset from [here](https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz) and extract it into the `resources` directory. We'll use fiftyone to load and preview the dataset.

In [ ]:
import fiftyone as fo

name = "google-military-vehicles"
dataset_dir = "../resources/dataset"

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.YOLOv4Dataset,
    name=name,
)

In [ ]:
session = fo.launch_app(dataset, auto=False)

## Train a yolov8 model with just the google dataset

To start with, we'll train a yolov8 model using only our newly created dataset. The following methods will split the dataset into train, test and val splits and export it into a folder in yolo format for training.

In [ ]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(
    dataset,
    {"train": 0.8, "val": 0.1, "test": 0.1}
)

In [ ]:
def export_yolo_data(
    samples, 
    export_dir, 
    classes, 
    label_field = "ground_truth", 
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples, 
                export_dir, 
                classes, 
                label_field, 
                split
            )   
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

In [ ]:
## export in YOLO format
export_yolo_data(
    dataset, 
    "google_vehicles", 
    ["AFV", "APC", "MEV", "LAV"], 
    split = ["train", "val", "test"]
)

We can now train our model. We'll use a `yolov8-large` model as our base and finetune it on 100 epochs.

In [ ]:
!yolo task=detect mode=train model=yolov8l.pt data=google_vehicles/dataset.yaml epochs=100 imgsz=640 batch=16

## Evaluating our model

The following will run inference on the test split and import the results into fiftyone to view the MAP.

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=google_vehicles/images/test save_txt=True save_conf=True

In [ ]:
# The test split of the dataset
test_view = dataset.match_tags("test")

In [ ]:
import os
import numpy as np
from tqdm import tqdm

def read_yolo_detections_file(filepath):
    detections = []
    if not os.path.exists(filepath):
        return np.array([])
    
    with open(filepath) as f:
        lines = [line.rstrip('\n').split(' ') for line in f]
    
    for line in lines:
        detection = [float(l) for l in line]
        detections.append(detection)
    return np.array(detections)

In [ ]:
def _uncenter_boxes(boxes):
    '''convert from center coords to corner coords'''
    boxes[:, 0] -= boxes[:, 2]/2.
    boxes[:, 1] -= boxes[:, 3]/2.

In [ ]:
def _get_class_labels(predicted_classes, class_list):
    labels = (predicted_classes).astype(int)
    labels = [class_list[l] for l in labels]
    return labels

In [ ]:
def convert_yolo_detections_to_fiftyone(
    yolo_detections, 
    class_list
    ):

    detections = []
    if yolo_detections.size == 0:
        return fo.Detections(detections=detections)
    
    boxes = yolo_detections[:, 1:-1]
    _uncenter_boxes(boxes)
    
    confs = yolo_detections[:, -1]
    labels = _get_class_labels(yolo_detections[:, 0], class_list) 
 
    for label, conf, box in zip(labels, confs, boxes):
        detections.append(
            fo.Detection(
                label=label,
                bounding_box=box.tolist(),
                confidence=conf
            )
        )

    return fo.Detections(detections=detections)

In [ ]:
def get_prediction_filepath(filepath, run_number = 1):
    run_num_string = ""
    if run_number != 1:
        run_num_string = str(run_number)
    filename = filepath.split("/")[-1].split(".")[0]
    return f"runs/detect/predict{run_num_string}/labels/{filename}.txt"

In [ ]:
def add_yolo_detections(
    samples,
    prediction_field,
    prediction_filepath,
    class_list
    ):

    prediction_filepaths = samples.values(prediction_filepath)
    yolo_detections = [read_yolo_detections_file(pf) for pf in prediction_filepaths]
    detections =  [convert_yolo_detections_to_fiftyone(yd, class_list) for yd in yolo_detections]
    samples.set_values(prediction_field, detections)

In [ ]:
filepaths = test_view.values("filepath")
prediction_filepaths = [get_prediction_filepath(fp) for fp in filepaths]
test_view.set_values(
    "yolov8l_det_filepath", 
    prediction_filepaths
)

add_yolo_detections(
    test_view, 
    "yolov8l", 
    "yolov8l_det_filepath", 
    ["AFV", "APC", "MEV", "LAV"]
)

In [ ]:
detection_results = test_view.evaluate_detections(
    "yolov8l", 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

In [ ]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

In [ ]:
detection_results.print_report()

## Run video tracking with our model

We can run video tracking of military vehicles using our trained model and the sample videos available in the `resources/test` directory.

In [ ]:
!cp runs/detect/train/weights/best.pt yolov8l-google.pt

In [ ]:
!python yolo_tracking/examples/track.py --yolo-model yolov8l-google.pt --reid-model mobilenetv2_x1_4_dukemtmcreid.pt --source ../resources/test/lav3.mp4 --save --project runs/track